 # Dziuk's method for mean curvature flow
 In this tutorial we solve an example of a so-called *geometric evolution problem*.
 Such problems are characterized by the fact that we not only need to solve
 a PDE on a manifold, but that this PDE solution itself determines how the manifold
 is evolving over time.
 Here, we consider the prototype problems of all geometric evolution problems,
 the mean the so-called mean curvature flow. Roughly speaking, a time-dependent surface geometry  $\Gamma_t$ follows the mean curvature flow, if
 the surface deforms in such way that the area integral
 $$ E(\Gamma_t) = \int_{\Gamma_t} 1 \;ds $$
 is minimized the fastest.
 Mean curvature flow is also an example of a curvature-driven interface evolution
 and such problems are ubiquitous in science and engineering and are used to
 describe problems such as
 - grain boundary motion in crystalline structures, e.g. snow flakes
 - phase separation problem (lava lamps, vinaigrette sauce)
 - tumor growth
 - shape optimization, e.g., optimization the shape of a wing to maximize drift/minimize drag)
 and many many more, see this [review article](https://link.springer.com/article/10.1365/s13291-013-0066-2),
 and [this one](https://www.cambridge.org/core/journals/acta-numerica/article/computation-of-geometric-partial-differential-equations-and-mean-curvature-flow/F2572D554B5D7CA83424E59F45D68EAE)
 for the plethora of approaches to solve such problems.

 ## Mathematical description
 Let $\Gamma$ be a
 closed oriented surface equipped with an outer normal vector field 
 $\nu$. 
 Then a measure for how much the surface bend in space is given by the surface gradient 
 of the normal field, 
 $$
 \Gamma  \ni x \mapsto \nabla_\Gamma\nu(x) .
 $$

This map is therefore also called the *Shape operator* (or *Weingarten map*).
 The mean curvature is simply defined by $H=\mathrm{tr}(\nabla_\Gamma\nu)$
 and corresponds to the sum of the largest and smallest nonvanishing eigenvalues of 
$\nabla_\Gamma\nu(x)$

For an evolving surface, we have the following picture in mind.
Starting from a reference/initial geometry configuration $\Gamma_0 = \widehat{\Gamma}$,
the surface geometry at time $t$ is given by a parametrization $X(\widehat{x},t)$
$$
\Gamma_0 \ni \widehat{x} \mapsto X(\widehat{x}, t) = x \in \Gamma_t.
$$

For the mean curvature flow, it can be show that
each point $X$ on the surface moves in normal direction of the surface 
with velocity $H\,\nu$, 
i.e. we have the equation
$$
\partial_t X = -H\,\nu.
$$

It can be show that mean curvature vector $-H\,\nu$ is related to the parametrization $X$ via the
Laplace-Beltrami problem
$$
H\,\nu=-\Delta_\Gamma X
$$
or weak form,
$$
\int_{\Gamma_t}\partial_t X\cdot v\,ds = -\int_{\Gamma_t}\nabla_{\Gamma_t}X:\nabla_{\Gamma_t} v\,ds \qquad \forall v\in H^1(\Gamma_t,\mathbb{R^3}).
$$
Using the mass and stiffness forms
$$
m_t(X,v)=\int_{\Gamma_t}X\cdot v\,ds,\qquad a_t(X,v):=\int_{\Gamma_t}\nabla_{\Gamma_t}X:\nabla_{\Gamma_t} v\,ds
$$
the equation reads
$$
m_t(\partial_tX,v)+a_t(X,v)=0.
$$


We use Dziuk's method for time discretization
$$
m_{t^n}(\frac{X^{n+1}-X^n}{\tau},v)+a_{t^n}(X^{n+1},v)=0.
$$

Note that when we integrate some $X(\cdot, s)$ over some surface $\Gamma_t$,
we have to think about $X(\cdot, s)$ as being reparametrized of $\Gamma_t$, i.e.
for $x = X(\widehat{x}, t) \in \Gamma_t$ we have
$$
X(x,s) = X(X(\widehat{x}, t), s)
$$
With this identification in mind, we realize that simply
$$
X(x,t) = x
$$
so $X(x,t)$ is simply the identity map $\vec{id}: x \mapsto x$ for  $x \in \Gamma_t$.

Now it is time to implement Dziuk's method in NGSolve. 
We start by defining the geometry and generating a mesh. 

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
R = 1
sphere = Sphere((0,0,0),R).faces[0]

# Mesh geometry with curved elements of order 2
order_g = 2
geo = OCCGeometry(sphere)

maxh = 0.15
mesh = Mesh(geo.GenerateMesh(maxh=maxh))
mesh.Curve(order_g)
Draw(mesh)

Making the ansatz for the sphere $\Gamma(t) = S_{R(t)}$ is a sphere of radius $R(t)$
one can derive and solve a simple ODE for the radius $r(t)$
to see that the evolution is described by
$$
R(t) = \sqrt{R_0^2 - 4t}
$$
So our geometry will get extinct at time $T_{ext} =\tfrac{R_0^2}{4}$

In [ ]:
# Radius of exact solution at time t
ex_R = lambda t : sqrt(R**2-4*t)
# Extinction time
Tend = R**2/4
# time step
tau = 2e-4

We define function spaces as well as grid functions describing 
- the current displacement ```dXh``` which will describe how to deform the initial mesh into $\Gamma_t$
- and the identity map ```Idh``` on $\Gamma_t$.

To draw, integrate etc. on a deformed domain $\Gamma_t$, we use the argument ```deformation=dXh```.

In [ ]:
order_p = 2
V = VectorH1(mesh, order=order_p)

# Displacement to apply deform initial mesh to final mesh
dXh = GridFunction(V)
dXh.vec[:] = 0

# Store identity mapping as grid function
Idh = GridFunction(V)
Idh.Set( CF( (x,y,z) ), definedon=mesh.Boundaries(".*"))

# Inital deformation
Xh = GridFunction(V)
Xh.Set( CF( (x,y,z) ), definedon=mesh.Boundaries(".*"))

# scene = Draw(Xh, mesh, deformation=Xh, vectors=True)
scene = Draw(Norm(dXh), mesh, deformation=dXh)
# TODO: vectors=True does not draw vectors on surface mesh?
# scene = Draw(CF((x,y,z)), mesh, deformation=Xh, vectors=True)
# norm_Xh = Norm(Xh)


Now we simply implement the weak form of Dziuk's method, with the little twist that we ask to integrate over the deformed domain.

In [ ]:
X,Y = V.TnT()

M = BilinearForm(V)
M += (X*Y + tau*InnerProduct(grad(X).Trace(), grad(Y).Trace()))*ds(deformation=dXh)
M.Assemble()
Minv = M.mat.Inverse(V.FreeDofs(), inverse="sparsecholesky")

l = LinearForm(V)
l += CF((x,y,z))*Y*ds(deformation=dXh)
l.Assemble()


Let's do the time-stepping. Here we use a) the ```Taskmanager``` which automatically can use several processor threads to solve our problem.
Moreover, we use the ```Update``` to update the inverse matrix computation.

In [ ]:
i = 0
t = 0
with TaskManager():
    while t <= Tend-tau:
        print ("\rt=", t, end="")
        M.Assemble()
        l.Assemble()
        Minv.Update()
        Xh.vec.data = Minv*l.vec
        dXh.vec.data = Xh.vec - Idh.vec
        
        if i % 10 == 0:
            scene.Redraw()
        
        t += tau
        i += 1

### Exercise

As a conclusion of this workshop, we ask you to wrap the above solver into a little python function
```python
def solve_mcf(mesh, order_p, tau, Tend):
    ...
```

and solve the MCF for a torus instead. You can use the function ```generate_torus_mesh``` from the ```helper module```.

**Hint** Don't let your torus evolve longer than $T\approx 0.1$